In [9]:
import sqlite3
import pandas as pd
import numpy as np

# ----------------------------
# 1. Generate energy data
# ----------------------------
np.random.seed(42)
num_rows = 1000

regions = ['North', 'South', 'East', 'West', 'Central']
facilities = ['Plant A', 'Plant B', 'Plant C']
machines = ['Boiler', 'Pump', 'Compressor', 'Conveyor']
dates = pd.date_range(start='2023-01-01', periods=num_rows, freq='D')

data = {
    'record_id': range(1, num_rows+1),
    'date': np.random.choice(dates, num_rows),
    'region': np.random.choice(regions, num_rows),
    'facility': np.random.choice(facilities, num_rows),
    'machine': np.random.choice(machines, num_rows),
    'energy_kwh': np.round(np.random.uniform(50, 500, size=num_rows), 2)  # fixed!
}

df = pd.DataFrame(data)

conn = sqlite3.connect("energy_database.db")
df.to_sql("energy_usage", conn, if_exists="replace", index=False)
print("✓ Energy usage database created")
df.head()

✓ Energy usage database created


,record_id,date,region,facility,machine,energy_kwh
0,1,2023-04-13,Central,Plant B,Pump,474.82
1,2,2024-03-11,Central,Plant B,Conveyor,116.96
2,3,2025-05-10,North,Plant B,Boiler,258.20
3,4,2023-09-28,North,Plant B,Conveyor,491.44
4,5,2023-04-17,South,Plant C,Pump,267.53


In [10]:
query1 = """
SELECT region,
       SUM(energy_kwh) AS total_energy,
       ROUND(AVG(energy_kwh), 2) AS avg_energy
FROM energy_usage
GROUP BY region
ORDER BY total_energy DESC;
"""
df_region = pd.read_sql_query(query1, conn)
df_region

,region,total_energy,avg_energy
0,North,58607.98,275.15
1,East,58086.61,280.61
2,South,56474.91,278.20
3,Central,56318.75,276.07
4,West,46626.91,269.52


In [11]:
query2 = """
SELECT facility,
       SUM(energy_kwh) AS total_energy,
       ROUND(AVG(energy_kwh), 2) AS avg_energy
FROM energy_usage
GROUP BY facility
ORDER BY total_energy DESC;
"""
df_facility = pd.read_sql_query(query2, conn)
df_facility

,facility,total_energy,avg_energy
0,Plant B,97045.31,279.67
1,Plant A,94927.97,275.95
2,Plant C,84141.88,272.30


In [12]:
# Query 3: Top 5 machines consuming the most energy
query3 = """
SELECT machine,
       SUM(energy_kwh) AS total_energy
FROM energy_usage
GROUP BY machine
ORDER BY total_energy DESC
LIMIT 5;
"""
df_machine = pd.read_sql_query(query3, conn)
df_machine

,machine,total_energy
0,Boiler,72771.57
1,Conveyor,70184.73
2,Compressor,67634.84
3,Pump,65524.02


In [13]:
# Query 4: Monthly energy consumption trend
query4 = """
SELECT strftime('%Y-%m', date) AS month,
       SUM(energy_kwh) AS total_energy
FROM energy_usage
GROUP BY month
ORDER BY month;
"""
df_monthly = pd.read_sql_query(query4, conn)
df_monthly

,month,total_energy
0,2023-01,7681.29
1,2023-02,10302.50
2,2023-03,5304.66
3,2023-04,10325.40
4,2023-05,9098.86
5,2023-06,8611.49
6,2023-07,9625.11
7,2023-08,8989.15
8,2023-09,9560.25
9,2023-10,6224.34


In [14]:
# Query 5: High energy usage records (above average)
query5 = """
SELECT record_id, region, facility, machine, energy_kwh, date
FROM energy_usage
WHERE energy_kwh > (SELECT AVG(energy_kwh) FROM energy_usage)
ORDER BY energy_kwh DESC
LIMIT 10;
"""
df_high_usage = pd.read_sql_query(query5, conn)
df_high_usage

,record_id,region,facility,machine,energy_kwh,date
0,921,Central,Plant B,Pump,499.50,2023-03-21 00:00:00
1,48,East,Plant A,Compressor,499.02,2023-07-09 00:00:00
2,926,East,Plant B,Compressor,498.93,2023-08-17 00:00:00
3,820,East,Plant B,Conveyor,498.73,2023-02-11 00:00:00
4,914,West,Plant A,Boiler,498.27,2024-07-09 00:00:00
5,962,North,Plant C,Pump,498.03,2023-03-31 00:00:00
6,308,West,Plant A,Pump,497.67,2023-05-03 00:00:00
7,355,North,Plant B,Pump,496.96,2025-08-03 00:00:00
8,255,East,Plant C,Conveyor,496.10,2023-08-13 00:00:00
9,950,South,Plant A,Boiler,495.81,2025-01-05 00:00:00
